### Using elastic search cloud client to store and search for data

In [7]:
import pandas as pd
import elasticsearch
from elasticsearch import Elasticsearch
from elasticsearch import helpers
data=pd.read_json('data.jsonl',lines=True)

In [8]:
data.columns

Index(['id', 'url', 'name', 'name_abbreviation', 'decision_date',
       'docket_number', 'first_page', 'last_page', 'citations', 'volume',
       'reporter', 'court', 'jurisdiction', 'frontend_url', 'preview',
       'casebody'],
      dtype='object')

In [9]:
import numpy as np
embed=np.load('embeddings.npy')     #loading previously stored embeddings

In [10]:
elk_data=data.to_dict("records")    #taking initial data and putting them into a dictionary

In [11]:
username="elastic"
password="pixEeaDbQi3Nrnj43HdI12mG"

In [12]:
import pandas as pd
final=pd.read_csv('final.csv')

In [13]:
final.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Processed,embeddings
0,0,0,0,['opinion coiírt action debt brought appellee ...,[-1.18838400e-01 4.82986346e-02 -2.54808320e-...
1,1,1,1,['opinion op court appeal conway circuit court...,[-1.18838400e-01 4.82986346e-02 -2.54808320e-...
2,2,2,2,['cross judge\nthe record case shows plaintiff...,[-1.18838400e-01 4.82986346e-02 -2.54808320e-...
3,3,3,3,['w hdub arnold chief justice\nthis case invol...,[-1.18838400e-01 4.82986346e-02 -2.54808320e-...
4,4,4,4,['ray thornton justice\nappellant brings appea...,[-1.18838400e-01 4.82986346e-02 -2.54808320e-...


In [14]:
# from sentence_transformers import SentenceTransformer
# class Tokenizer(object):
#     def __init__(self):
#         self.model = SentenceTransformer('all-MiniLM-L6-v2')

#     def get_token(self, documents):
#         sentences  = [documents]
#         sentence_embeddings = self.model.encode(sentences)
#         encod_np_array = np.array(sentence_embeddings)
#         encod_list = encod_np_array.tolist()
#         return encod_list[0]
# i=Tokenizer()
# p="Hello"
# i.get_token(p)

In [15]:
len(embed)

59735

In [16]:
for x in elk_data:
    print(x.get("citations", "")[0]['cite'])
    print(embed[0])
    break

9 F. Cas. 137
[-3.09154280e-02  7.34250396e-02 -5.35513088e-02 -5.10694198e-02
 -7.19274506e-02  5.55797406e-02  4.23816592e-02  4.53834832e-02
 -3.30751538e-02  4.23280783e-02  7.22314417e-02 -1.96809899e-02
  5.42776212e-02  2.84021255e-03 -6.59408001e-03  5.66306412e-02
  7.97524899e-02  2.64791064e-02 -9.05744452e-03  3.90769690e-02
 -5.06056696e-02  5.02215028e-02 -4.51204665e-02 -7.80457957e-03
 -2.77637988e-02 -3.35983746e-02 -6.12758324e-02 -3.04323621e-04
  4.71183844e-02 -9.07716006e-02 -3.11128087e-02  1.46439329e-01
 -1.78255271e-02  4.92394865e-02  1.38864052e-02  4.28559035e-02
 -1.37360487e-02 -7.19317198e-02 -4.20686379e-02 -1.60640981e-02
 -3.17895189e-02 -1.81900933e-02 -4.24116626e-02 -8.72379839e-02
 -1.81187168e-02 -5.48724737e-03 -4.21734862e-02 -3.68014537e-02
 -3.79192457e-02  3.79123986e-02 -3.78774032e-02 -3.14938091e-02
 -1.48426313e-02  3.01201604e-02 -4.77582924e-02  1.59844309e-02
 -2.47888770e-02  1.02014251e-01 -2.77355053e-02 -2.29747240e-02
  1.6669772

initialising elastic search client

In [17]:
from elasticsearch import Elasticsearch
cloud_id="My_deployment:YXNpYS1zb3V0aDEuZ2NwLmVsYXN0aWMtY2xvdWQuY29tJGNlMzFlZTExMGY1NjQzNDI4NTMwOWE4ZjdhNTI4OTUxJGYwOWRkN2YxMDlmMDQ2MGFhY2M2ZDI5MjZkNDAwZjhj"
es_client = Elasticsearch( cloud_id=cloud_id, 
                  http_auth=(username, password),)                  

/tmp/ipykernel_407/3157966985.py:3: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es_client = Elasticsearch( cloud_id=cloud_id,


Tried to use elastic search query with script_score to get cosine similarity but it did not work due to change in elastic search versions, this would work in elastic search 7 but due to our cluster being of elastic search 8 , we used something else

In [18]:
# def get_res(query_vector):    
#     s_body = {
#             "query": {
#                 "script_score": {
#                     "query": {
#                         "match_all": {}
#                     },
#                     "script": {
#                         "source": "cosineSimilarity(params.query_vector, 'embeddings') + 1.0",
#                         "params": {"query_vector": query_vector}
#                     }
#                 }
#             }
#         }
#     f=es_client.search(index=["internhsip1"],script_fields=s_body['query'])
#     return f
# from sentence_transformers import SentenceTransformer
# model=SentenceTransformer('all-MiniLM-L6-v2')
# query=model.encode("The defendant was found to be guilty on the charges of theft")
# query=query.tolist()
# q=get_res(query)

### Created mappings to store data into index: Internship1 , the fields we are using are : id, url , name , abbreviated_name , decision_data , citations , casebody , processed_casebody_data (the data that was preprocessed by us) and the embeddings with a similarity parameter of cosine as we are using semantic searching.

We also initialised the cluster with 2 shards so the data we can store in this cluster is approximately around 20 gb or so , along with one replica in case of mishap

In [19]:
# config = {
#     "mappings": {
#         "properties": {
#             "id": {"type": "integer"},
#             "url": {"type": "text"},
#             "name":{"type":"text"},
#             "name_abbv":{"type":"text"},
#             "decision_date":{"type":"date"},
#             "citations":{"type":"text"},
#             "casebody":{"type":"text"},
#             "processed_casebody_data":{"type":"text"},
#             "embeddings": {
#                     "type": "dense_vector",
#                     "dims": 384,
#                     "index":True,
#                     "similarity":"cosine"
#                 }
#             }
#     },
#     "settings": {
#         "number_of_shards": 2,
#         "number_of_replicas": 1
#     }
# }

# es_client.indices.create(
#     index="internship1",
#     body=config
# )

#check if the index has been created successfully
print(es_client.indices.exists(index=["internship1"]))

True


Storing data into the cluster, we use a try except block to be able to use those missing values mentioned before

In [20]:
j=-1
for x in elk_data:
    try:
        j+=1
        _={
           "id":x.get("id", ""),
            "url":x.get("url", ""),
            "name":x.get("name", ""),
            "name_abbv":x.get("name_abbreviation", ""),
            "decision_date":x.get("decision_date", ""),
            "citations":x.get("citations", "")[0]['cite'],
            "casebody":x.get("casebody", "")['data']['opinions'][0]['text'],
            "processed_casebody_data":final['Processed'][j],
            "embeddings":embed[j],
        }
        es_client.index(index='internship1',body=_)
    except Exception as e:
        _={
           "id":x.get("id", ""),
            "url":x.get("url", ""),
            "name":x.get("name", ""),
            "name_abbv":x.get("name_abbreviation", ""),
            "decision_date":x.get("decision_date", ""),
            "citations":x.get("citations", "")[0]['cite'],
            "casebody":x.get("casebody", "")['data']['head_matter'],
            "processed_casebody_data":final['Processed'][j],
            "embeddings":embed[j],
        }
        es_client.index(index='internship1',body=_)
        print(e)


/tmp/ipykernel_407/479776050.py:16: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es_client.index(index='internship1',body=_)
/tmp/ipykernel_407/479776050.py:29: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es_client.index(index='internship1',body=_)


list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range


In [21]:
j

59734

##### Now, finally we can use K-nearest neighbour search on our data to be able to see which values are closest to our required data witht the help of cosine similarity

In [99]:
import re
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
def preprocess(query):
        s=""
        lmt=WordNetLemmatizer()
        query=query.lower()
        sentence=lmt.lemmatize(query)
        # print(sentence)
        for x in sentence.split(" "):
            if x not in stopwords.words('english'):
                s+=x+" "
        s = re.sub(r'[^\w\s]','',s)
        return s

def search(query):
    from sentence_transformers import SentenceTransformer
    model=SentenceTransformer('all-MiniLM-L6-v2')
    query=preprocess(query)           #preprocessing query entered
    query=model.encode(query)
    body={
      "knn": {
        "field": "embeddings",
        "query_vector": query,
        "k": 1,                   #looking for the K closest vectors to the input query 
        "num_candidates": 500    #increase in num_candidates helps make better predictions with the downside of time increase but with our data size its not much of a problem
      },
      "fields": [ "casebody" ]
    }
    f=es_client.knn_search(index='internship1',body=body)   #using knn search
    return f

Checking if search works

In [94]:
if es_client.ping():
    query=input()
    pp=search(query)
    print(pp['hits']['hits'][0]['fields'])

/tmp/ipykernel_407/4144933718.py:30: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  f=es_client.knn_search(index='internship1',body=body)   #using knn search
/tmp/ipykernel_407/4144933718.py:30: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  f=es_client.knn_search(index='internship1',body=body)   #using knn search


{'casebody': ['Ward, J.\nAppellants were charged by information with an assault with intent to kill. The material part of the information reads, as follows:\n“The said defendants on or about tbe 15th day of January, 1952, in Benton County, Arkansas, did unlawfully, willfully, feloniously, deliberately and with malice aforethought assault Bay Harrison with a deadly weapon, to-wit: An Iron Bod about one and one-half foot long, being then and there held in the hands of Jack Sharpen-steen, Sr., with the unlawful and felonious intent then and there to kill and murder the said Bay Harrison, against the peace and dignity of the State of Arkansas. ’ ’\nOn March 20, 1952, appellants were tried in circuit court on the above information and convicted by a jury of assault and battery.\nAppellants prosecute this appeal on the sole ground that, under the former decisions of this court, they cannot be legally convicted of assault and battery under the information set out above. This is no doubt the l

In [95]:
pp['hits']['hits'] #complete hit infromation from search

[{'_index': 'internship1',
  '_id': 'ABEmi4gBzDogf52k9B8A',
  '_score': 0.7329025,
  '_source': {'id': 1660087,
   'url': 'https://api.capapi.org/v1/cases/1660087/',
   'name': 'Sharpensteen v. State',
   'name_abbv': 'Sharpensteen v. State',
   'decision_date': '1952-07-07',
   'citations': '220 Ark. 839',
   'casebody': 'Ward, J.\nAppellants were charged by information with an assault with intent to kill. The material part of the information reads, as follows:\n“The said defendants on or about tbe 15th day of January, 1952, in Benton County, Arkansas, did unlawfully, willfully, feloniously, deliberately and with malice aforethought assault Bay Harrison with a deadly weapon, to-wit: An Iron Bod about one and one-half foot long, being then and there held in the hands of Jack Sharpen-steen, Sr., with the unlawful and felonious intent then and there to kill and murder the said Bay Harrison, against the peace and dignity of the State of Arkansas. ’ ’\nOn March 20, 1952, appellants were tr

CHECKING METRICS WITH MANUALLY INPUTTED QUERIES

In [108]:
answer=[]
question=[]
for x in range(10):
    if es_client.ping():
        query=input()
        question.append(query)
        pp=search(query)
        answer.append(pp['hits']['hits'][0]['fields']['casebody'])

/tmp/ipykernel_407/2983719991.py:30: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  f=es_client.knn_search(index='internship1',body=body)   #using knn search
/tmp/ipykernel_407/2983719991.py:30: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  f=es_client.knn_search(index='internship1',body=body)   #using knn search


In [124]:
question_answer=pd.DataFrame({"question":question,"answers": answer})

In [128]:
question_answer.to_csv('results.csv')

In [129]:
answer[1]

['Karen R. Baker, Judge.\nThis is an administrative appeal from the Arkansas Professional Bail Bond Company and Professional Bail Bondsman Licensing Board. The appellant, Elizabeth Frawley, is a licensed bail bondsman who was employed by appellant, J & J Bonding, Inc., on December 16, 1997. Appellant Frawley was notified by the Board that she solicited business or advertised for business in or about a place where prisoners were confined in violation of Ark. Code Ann. section 17-19-105(2). Appellant J & J Bonding, Inc. was notified that it was responsible for Ms. Frawley’s actions pursuant to Ark. Code Ann. section 17-19-210(b) because she was acting within the scope of her authority as a J & J bondsman. The statutory prohibition against solicitation reads as follows:\nNo professional bail bondsman or professional bail bond company, nor court, nor law enforcement officer nor any individual working on behalf of a professional bail bondsman or professional bail bond company shall: . . .(2

#### All Data Has relevance to input query ***but further testing is require with more queries.***